In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_hub as hub
import sqlite3
import pandas as pd
import numpy as np
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
!test -f newsticker2019.db || wget https://datanizing.com/bert/heise/newsticker2019.db

In [ ]:
sql = sqlite3.connect("newsticker2019.db")
texts = pd.read_sql("SELECT id, text FROM news", sql, index_col="id")

In [ ]:
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

In [ ]:
def elmo_vectors(x):
  embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [ ]:
from tqdm.auto import trange
vecs = []
for i in trange(int(len(texts)/100)+1):
    r = elmo_vectors(texts[i*100:(i+1)*100].text.map(str).values)
    if len(vecs) > 0:
        vecs = np.vstack([vecs, r])
    else:
        vecs = r

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# Cosinus-Ähnlichkeit aller Dokumente miteinander berechnen
sim = cosine_similarity(vecs, vecs)

In [ ]:
# Ergebnis ist eine Matrix
sim.shape

In [ ]:
# Größter Wert wird für identische Meldungen gefunden
sim.max()

In [ ]:
for threshhold in [0.7, 0.8, 0.9, 0.95]:
    s = sim.copy()
    # alle über dem Threshhold ausmaskieren
    s[s > threshhold] = 0
    (max_a, max_b) = np.unravel_index(np.argmax(s), s.shape)
    print("%f '%s' '%s'" % (threshhold, texts.iloc[max_a]["text"], texts.iloc[max_b]["text"]))

In [ ]:
# Alternative Aufgabe: Ähnlichste Nachrichten in unterschiedlichen Monaten finden